In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.utils import *
from sklearn.preprocessing import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [ ]:
X = pd.read_csv('X_train.csv', encoding='cp949')
y = pd.read_csv('y_train.csv', encoding='cp949')

#불필요한 칼럼 삭제
X2 = X.drop('년월일시분', axis = 1)

In [ ]:
y

,청담대교 수위 (단위: cm),한강대교 수위 (단위: cm),행주대교 수위 (단위: cm)
0,310.7,290.0,275.3
1,314.7,290.0,275.3
2,313.7,290.0,275.3
3,311.7,290.0,276.3
4,311.7,291.0,277.3
...,...,...,...
210669,281.7,278.0,271.3
210670,279.7,278.0,272.3
210671,278.7,277.0,272.3
210672,277.7,276.0,271.3


In [ ]:
# test = pd.read_csv('X_test.csv', encoding='cp949')
# test.isnull().sum()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.2, random_state = 1234)

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
X_train = np.asarray(X_train.values.tolist(), dtype=np.float64)
X_test = np.asarray(X_test.values.tolist(), dtype=np.float64)
print(X_train.shape, X_test.shape)

(168539, 14) (42135, 14)


In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)
print(y_train.shape, y_test.shape)

(168539, 3) (42135, 3)


In [ ]:
y_train

array([[321.7, 313. , 301.3],
       [291.7, 279. , 270.3],
       [307.7, 292. , 281.3],
       ...,
       [294.7, 285. , 276.3],
       [289.7, 276. , 263.3],
       [287.7, 284. , 270.3]])

MinMaxScaler 후 모델링 엄격한 검증을 위해서 트레인 셋만 스케일링 해준다.

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
y_train = scaler.fit_transform(y_train)

다시 3차원으로 바꿔줘서 LSTM 모델로 넣어준다.

In [ ]:
X_train = X_train.reshape(-1, 1, 14)
X_test = X_test.reshape(-1, 1, 14)

y_train = y_train.reshape(-1, 1, 3)
y_test = y_test.reshape(-1, 1, 3)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(168539, 1, 14) (42135, 1, 14) (168539, 1, 3) (42135, 1, 3)


In [ ]:
y_train

array([[[0.375     , 0.39215686, 0.46153846]],

       [[0.17763158, 0.16993464, 0.22307692]],

       [[0.28289474, 0.25490196, 0.30769231]],

       ...,

       [[0.19736842, 0.20915033, 0.26923077]],

       [[0.16447368, 0.1503268 , 0.16923077]],

       [[0.15131579, 0.20261438, 0.22307692]]])

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(1, 14,), return_sequences=True, activation='relu'))
#model.add(Dense(5))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(3))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00008)
model.compile(loss='mse', optimizer=optimizer, metrics=['acc'])
early_stop = EarlyStopping(monitor='val_loss', patience=10)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 256)            277504    
                                                                 
 lstm_1 (LSTM)               (None, 1, 256)            525312    
                                                                 
 dense (Dense)               (None, 1, 128)            32896     
                                                                 
 dense_1 (Dense)             (None, 1, 64)             8256      
                                                                 
 dense_2 (Dense)             (None, 1, 3)              195       
                                                                 
Total params: 844,163
Trainable params: 844,163
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(X_train.shape)
hist = model.fit(X_train, y_train, epochs=100, batch_size=24, validation_split=0.2,callbacks=[early_stop])

(168539, 1, 14)
Epoch 1/100
5618/5618 [==============================] - 81s 14ms/step - loss: 0.0025 - acc: 0.9183 - val_loss: 2.8229e-04 - val_acc: 0.9736
Epoch 2/100
5618/5618 [==============================] - 78s 14ms/step - loss: 2.6433e-04 - acc: 0.9747 - val_loss: 2.4492e-04 - val_acc: 0.9810
Epoch 3/100
5618/5618 [==============================] - 78s 14ms/step - loss: 2.4326e-04 - acc: 0.9757 - val_loss: 2.3537e-04 - val_acc: 0.9791
Epoch 4/100
5618/5618 [==============================] - 79s 14ms/step - loss: 2.2652e-04 - acc: 0.9751 - val_loss: 2.2153e-04 - val_acc: 0.9701
Epoch 5/100
5618/5618 [==============================] - 80s 14ms/step - loss: 2.1400e-04 - acc: 0.9765 - val_loss: 2.0642e-04 - val_acc: 0.9808
Epoch 6/100
5618/5618 [==============================] - 80s 14ms/step - loss: 2.0792e-04 - acc: 0.9774 - val_loss: 2.0894e-04 - val_acc: 0.9790
Epoch 7/100
5618/5618 [==============================] - 80s 14ms/step - loss: 2.0479e-04 - acc: 0.9771 - val_loss: 1.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
y_pred = model.predict(X_test)
print(type(y_pred))
y_pred = y_pred.reshape(-1, 3)
y_test = y_test.reshape(-1, 3)

# # Linear Regression(선형회귀) 사용

# lr = LinearRegression()

# lr.fit(X_train, y_train)

# pred = lr.predict(X_test)

# print('r2 : ', r2_score(y_test, y_pred)) #결정계수
print('mse : ', mean_squared_error(y_test, y_pred)) # mse
print('rmse : ', np.sqrt(mean_squared_error(y_test, y_pred)))

# plt.scatter(y_test, pred)

<class 'numpy.ndarray'>
mse :  161174.82164986964
rmse :  401.46584119931003


1.MinMaxScale은 전혀 효과가 없었다. 여기서 스케일링은 트레인셋만 진행하였다. (왜냐면 우리가 공모전의 정답지를 맞춰야할 입장이기 때문에, 튜토리얼이나 예제문제처럼 테스트셋을 minmaxScale하거나 트레인셋의 평균 분산값으로 Normalize할 수 없기 때문이다.) y-target 값은 0과1 사이에 있는 값이 아니기 때문에, minmaxscale로는 부적절하다.

In [ ]:
# y_pred = model.predict(X_test)
np.isnan(y_pred).sum()

0

In [ ]:
y_pred

array([[274.8823 , 267.65076, 274.02533],
       [299.50684, 280.30743, 258.90125],
       [356.38452, 340.9192 , 324.48538],
       ...,
       [341.13202, 327.95087, 314.27872],
       [296.8855 , 284.52847, 278.2618 ],
       [292.81555, 279.58298, 272.17087]], dtype=float32)

In [ ]:
y_test

array([[276.7, 273. , 276.3],
       [299.7, 276. , 259.3],
       [357.7, 341. , 326.3],
       ...,
       [342.7, 335. , 316.3],
       [298.7, 284. , 280.3],
       [294.7, 278. , 274.3]])

In [ ]:
zdef plot_history(hist):
    for key, values in hist.items():
        plt.plot(values)
        plt.title(key)
        plt.xlabel('epoch')
        plt.show()

plot_history(hist.history)

SyntaxError: ignored

LSTM 5개의 계층, Batch size 24, learning rate 0.00008에서 가장 낮은 RMSE(2.429)가 나왔습니다. 이후 파라미터 개선 및 Minmaxscaler 작업 실시 예정.

RobustScaler로 전처리 후 모델

In [ ]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
y_train = scaler.fit_transform(y_train)

In [ ]:
X_train = X_train.reshape(-1, 1, 14)
X_test = X_test.reshape(-1, 1, 14)

y_train = y_train.reshape(-1, 1, 3)
y_test = y_test.reshape(-1, 1, 3)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(168539, 1, 14) (42135, 1, 14) (168539, 1, 3) (42135, 1, 3)


In [ ]:
hist = model.fit(X_train, y_train, epochs=100, batch_size=24, validation_split=0.2)

Epoch 1/100
5618/5618 [==============================] - 82s 14ms/step - loss: 0.0250 - acc: 0.7254 - val_loss: 0.0051 - val_acc: 0.7518
Epoch 2/100
5618/5618 [==============================] - 77s 14ms/step - loss: 0.0047 - acc: 0.7816 - val_loss: 0.0050 - val_acc: 0.7958
Epoch 3/100
5618/5618 [==============================] - 78s 14ms/step - loss: 0.0040 - acc: 0.7956 - val_loss: 0.0037 - val_acc: 0.7997
Epoch 4/100
5618/5618 [==============================] - 77s 14ms/step - loss: 0.0037 - acc: 0.8037 - val_loss: 0.0035 - val_acc: 0.8019
Epoch 5/100
5618/5618 [==============================] - 78s 14ms/step - loss: 0.0035 - acc: 0.8092 - val_loss: 0.0036 - val_acc: 0.8186
Epoch 6/100
5618/5618 [==============================] - 78s 14ms/step - loss: 0.0034 - acc: 0.8111 - val_loss: 0.0033 - val_acc: 0.8083
Epoch 7/100
5618/5618 [==============================] - 77s 14ms/step - loss: 0.0033 - acc: 0.8137 - val_loss: 0.0032 - val_acc: 0.8131
Epoch 8/100
5618/5618 [==================

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
y_pred = model.predict(X_test)
print(type(y_pred))
y_pred = y_pred.reshape(-1, 3)
y_test = y_test.reshape(-1, 3)

# # Linear Regression(선형회귀) 사용

# lr = LinearRegression()

# lr.fit(X_train, y_train)

# pred = lr.predict(X_test)

# print('r2 : ', r2_score(y_test, y_pred)) #결정계수
print('mse : ', mean_squared_error(y_test, y_pred)) # mse
print('rmse : ', np.sqrt(mean_squared_error(y_test, y_pred)))

# plt.scatter(y_test, pred)

<class 'numpy.ndarray'>
mse :  254966.30366726444
rmse :  504.9418814747539


결론: 그 뿐만 아니라, 머신러닝을 사용한 scale 자체가 부적절하다는게 의견입니다. 애초에 공모전 특성상 테스트셋을 건드릴 수 없어서, sklearn을 이용한 기계적인 scaling 하는거 자체가 무의미합니다. 우리가 자체적으로 전처리를 잘해서 스코어를 올리는걸 목표로 삼는게 좋다는게 의견입니다.